# 1pCPU - A small 8 bit CPU in MyHDL

This is a fork (for synthesis experiments) of:

[https://github.com/pcornier/1pCPU](https://github.com/pcornier/1pCPU)

In [8]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [9]:
import sys
sys.path.insert(0, '../../..')

In [10]:
# OPCODES
# instruction:     00000---
# addressing mode: -----000
from collections import namedtuple

Adm = namedtuple('Adm', ['IMP', 'IMM', 'ABS', 'REL', 'IDX'])
Opc = namedtuple('Opc', [
    'LDA', 'STA', 'PHA', 'PLA', 'ASL', 'ASR', 'TXA', 'TAX',
    'INX', 'DEX', 'ADD', 'SUB', 'AND', 'OR', 'XOR', 'CMP',
    'RTS', 'JNZ', 'JZ', 'JSR', 'JMP'
])
opc = Opc(*range(21))
adm = Adm(*range(5))
                          
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=[*opc._asdict()])
for n,i in adm._asdict().items():
    df.loc[n] = (np.left_shift(pd.Series(opc._asdict()),3) + i).apply(hex)

allowed = [0x1, 0x2, 0x4, 0xa, 0xc, 0x10, 0x18, 0x21, 0x29, 0x30, 0x38, 0x40, 0x48, 0x51, 0x59, 0x61, 0x69, 0x71, 0x79, 0x80, 0x8b, 0x93, 0x9a, 0xa2]
df.style.applymap(lambda v: 'background-color: #f55' if not int(v, 16) in allowed else 'background-color: #5f5')


,LDA,STA,PHA,PLA,ASL,ASR,TXA,TAX,INX,DEX,ADD,SUB,AND,OR,XOR,CMP,RTS,JNZ,JZ,JSR,JMP
IMP,0x0,0x8,0x10,0x18,0x20,0x28,0x30,0x38,0x40,0x48,0x50,0x58,0x60,0x68,0x70,0x78,0x80,0x88,0x90,0x98,0xa0
IMM,0x1,0x9,0x11,0x19,0x21,0x29,0x31,0x39,0x41,0x49,0x51,0x59,0x61,0x69,0x71,0x79,0x81,0x89,0x91,0x99,0xa1
ABS,0x2,0xa,0x12,0x1a,0x22,0x2a,0x32,0x3a,0x42,0x4a,0x52,0x5a,0x62,0x6a,0x72,0x7a,0x82,0x8a,0x92,0x9a,0xa2
REL,0x3,0xb,0x13,0x1b,0x23,0x2b,0x33,0x3b,0x43,0x4b,0x53,0x5b,0x63,0x6b,0x73,0x7b,0x83,0x8b,0x93,0x9b,0xa3
IDX,0x4,0xc,0x14,0x1c,0x24,0x2c,0x34,0x3c,0x44,0x4c,0x54,0x5c,0x64,0x6c,0x74,0x7c,0x84,0x8c,0x94,0x9c,0xa4


In [11]:
# ROM source
src = """
; testing some basic instructions
lda #$0
tax
_1:
sta range,x
inx
txa
cmp #$10
jnz _1
at $100
range db $0

"""

In [12]:
# Assembler

from pyparsing import *

class Parser():

    pos = 0
    labels = {}
    binary = []

    def setOrig(self, tokens):
        self.pos = int(tokens.adr, 16)

    def setLabel(self, tokens):
        self.labels[tokens.lbl[0]] = self.pos-1

    def setRef(self, tokens):
        self.labels[tokens.lbl] = self.pos
        if tokens.size == 'db':
            self.binary.append(int(tokens.val, 16))
            self.pos += 1
        else:
            self.binary.append(int(tokens.val, 16) & 0xff)
            self.binary.append(int(tokens.val, 16) >> 8)
            self.pos += 2

    def setOp(self, tokens):
        c = getattr(opc, tokens.op) << 3
        if tokens.op == 'LDA':
            if tokens.am == '#$':
                c = c | adm.IMM
                self.pos += 2
                self.binary.append(c)
                self.binary.append(int(tokens.val, 16))
            elif tokens.am == '$':
                c |= adm.IDX if tokens.idx else adm.ABS
                self.pos += 3
                self.binary.append(c)
                l = int(tokens.val, 16) & 0xff
                h = int(tokens.val, 16) >> 8
                self.binary.append(h)
                self.binary.append(l)
            elif tokens.lbl:
                c |= adm.IDX if tokens.idx else adm.ABS
                self.pos += 3
                self.binary.append(c)
                self.binary.append(tokens.lbl)
        elif tokens.op == 'STA':
            c |= adm.IDX if tokens.idx else adm.ABS
            if tokens.am == '$':
                self.pos += 3
                self.binary.append(c)
                l = int(tokens.val, 16) & 0xff
                h = int(tokens.val, 16) >> 8
                self.binary.append(h)
                self.binary.append(l)
            elif tokens.lbl:
                self.pos += 3
                self.binary.append(c)
                self.binary.append(tokens.lbl[0])
        elif tokens.op in ['TAX', 'TXA', 'PHA', 'PLA', 'RTS', 'ASL', 'ASR', 'INX', 'DEX']:
            c = c | adm.IMP
            self.pos += 1
            self.binary.append(c)
        elif tokens.op in ['ADD', 'SUB', 'AND', 'OR', 'XOR', 'CMP']:
            c = c | adm.IMM
            self.pos += 2
            self.binary.append(c)
            self.binary.append(int(tokens.val, 16))
        elif tokens.op in ['JNZ', 'JZ']:
            c = c | adm.REL
            self.pos += 2
            self.binary.append(c)
            self.binary.append(tokens.lbl[0])
        elif tokens.op in ['JSR', 'JMP']:
            c = c | adm.ABS
            self.pos += 3
            self.binary.append(c)
            self.binary.append(tokens.lbl[0])
        
    def parse(self, src):
        
        op = oneOf(' '.join(opc._asdict()), caseless=True)
        org = (Word('atAT') + '$' + Word(hexnums)('adr')).setParseAction(self.setOrig)
        val = Word('#$')('am') + Word(hexnums)('val')
        lbl = (~op + Word(alphanums + '_'))('lbl')
        prm = (val|lbl) + ~Literal(':') + Optional(Word(', x'))('idx')
        label = (lbl + ':').setParseAction(self.setLabel)
        comment = ';' + restOfLine
        instruction = (op('op') + Optional(prm('prm'))).setParseAction(self.setOp)
        ref = (Word(alphanums + '_')('lbl') + oneOf('db dw')('size') + '$' + Word(hexnums)('val')).setParseAction(self.setRef)
        asm = OneOrMore(org | label | instruction | comment | ref)

        asm.parseString(src, parseAll=True)

        for i,b in enumerate(self.binary):
            if not isinstance(b, int):
                if b in self.labels.keys():
                    if self.binary[i-1] in [0x8b, 0x93]: #todo extract addressing mode
                        self.binary[i] = self.labels[b] - i + 1
                    else:
                        self.binary[i] = self.labels[b] & 0xff
                        self.binary.insert(i+1, self.labels[b] >> 8)
                else:
                    print('unresolved', b)

        return tuple(self.binary)

rom = Parser().parse(src)

print('Compiled ROM: ', end='')
for b in rom:
    if b < 0: b = 0x100 + b
    print(f'{b:02x}', end=' ')


Compiled ROM: 01 00 38 0c 00 01 40 30 79 10 8b f8 00 

In [13]:
# RAM & CPU

from myirl.emulation.myhdl import *


# 2k RAM + ROM
@block
def mem(clk, adr, we, di, do):
    
    ram = [Signal(intbv(0)[8:]) for i in range(0x2000)] # 8k

    @always(clk.posedge)
    def logic():
        if we:
            ram[adr.val].next = di
        else:
            if adr < len(rom):
                do.next = rom[adr.val]
            else:
                do.next = ram[adr.val]
        
    return logic

@block
def processor(clk, rst, data_in, data_out, adr, we):

    """
    IR = instruction register
    IM = immediate value
    RX = X register
    RW = W register used for status flags
    SR = status register
    AM = addressing mode
    SP = stack pointer
    """
    
    # (F1, F2, D, E, M1, M2) = range(0,6)
    s = enum('F1', 'F2', 'D', 'E', 'M1', 'M2')
    pc, incpc, incpc2, jumppc = [ Signal(modbv(0)[11:]) for _ in range(4) ]
    cyc = Signal(s.F1)
    ir, im, ra, rx, rw, sr, am = (Signal(modbv(0)[8:]) for _ in range(7))
    incim = Signal(modbv(0)[8:])
    sp, pushsp, popsp = [ Signal(modbv(0xff)[8:]) for _ in range(3) ]

    @always_comb
    def assign():
        incim.next = im + rx
        incpc.next = pc + 1
        incpc2.next = pc + 2
        pushsp.next = sp - 1
        popsp.next = sp + 1
        jumppc.next = pc + im.signed()
    
    @always_seq(clk.posedge, rst)
    def logic():    
        
        if cyc == s.F1:
            adr.next = incpc
            pc.next = incpc
            cyc.next = s.F2

        elif cyc == s.F2:
            adr.next = incpc
            ir.next = data_out
            cyc.next = s.D

        elif cyc == s.D:
            im.next = data_out
            am.next = ir & 7
            ir.next = ir[:3]
            if ir[:3] == opc.RTS: # rts
                adr.next = popsp
                sp.next = popsp  
            cyc.next = s.E

        elif cyc == s.E:
            if ir == opc.LDA: # lda
                if am == adm.IMM:
                    ra.next = im
                    pc.next = incpc
                elif am == adm.ABS:
                    adr.next = concat(data_out, im)
                    pc.next = incpc2
                elif am == adm.IDX:
                    adr.next = concat(data_out, incim)
                    pc.next = incpc2
            elif ir == opc.STA: # sta
                if am == adm.ABS:
                    adr.next = concat(data_out, im)
                    we.next = 1
                    data_in.next = ra
                    pc.next = incpc2
                elif am == adm.IDX:
                    adr.next = concat(data_out, incim)
                    we.next = 1
                    data_in.next = ra
                    pc.next = incpc2
            elif ir == opc.TAX: # tax
                rx.next = ra
                rw.next = 1
            elif ir == opc.TXA: # txa
                ra.next = rx
            elif ir == opc.ADD: # add im
                ra.next = ra + im
                pc.next = incpc
            elif ir == opc.SUB: # sub im
                ra.next = ra - im
                pc.next = incpc
            elif ir == opc.AND: # and im
                ra.next = ra & im
                pc.next = incpc
            elif ir == opc.OR: # or im
                ra.next = ra | im
                pc.next = incpc
            elif ir == opc.XOR: # xor im
                ra.next = ra ^ im
                pc.next = incpc
            elif ir == opc.ASL: # asl im
                ra.next = ra << im
            elif ir == opc.ASR: # asr im
                ra.next = ra >> im
            elif ir == opc.JNZ: # jnz rel
                if sr[6] == 0:
                    pc.next = jumppc
                else:
                    pc.next = incpc
            elif ir == opc.JZ: # jz rel
                if sr[6] != 0:
                    pc.next = jumppc
                else:
                    pc.next = incpc
            elif ir == opc.INX: # inx
                rx.next = rx + 1
                rw.next = 1
            elif ir == opc.DEX: # dex
                rx.next = rx - 1
                rw.next = 1
            elif ir == opc.PHA: # pha
                adr.next = sp
                sp.next = pushsp
                data_in.next = ra
                we.next = 1
            elif ir == opc.PLA: # pla
                sp.next = popsp
                adr.next = popsp
            elif ir == opc.CMP: # cmp im
                rw.next = 2
                sr.next = concat((ra-im)>=0x80, (ra-im)==0, sr[6:0])
                pc.next = incpc
            elif ir == opc.JSR: # jsr abs
                adr.next = sp
                sp.next = pushsp
                data_in.next = incpc2[11:8]
                we.next = 1
            elif ir == opc.RTS: # rts
                adr.next = popsp
                sp.next = popsp
            elif ir == opc.JMP: # jmp abs
                pc.next = concat(data_out[3:], im)
            cyc.next = s.M1

        elif cyc == s.M1:
            if (ir == opc.PLA) or (ir == opc.LDA and am == adm.ABS or am == adm.IDX):
                ra.next = data_out
            elif ir == opc.JSR:
                adr.next = sp
                sp.next = pushsp

                data_in.next = incpc2[8:]
                we.next = 1
                pc.next = concat(data_out[3:], im)
            elif ir == opc.RTS:
                pc.next = data_out
            else:
                we.next = 0
                adr.next = pc
            cyc.next = s.M2

        elif cyc == s.M2:
            if ir == 0x11:
                ra.next = data_out
                sr.next = concat(data_out>=0x80, data_out==0, sr[6:0])
            elif rw == 0:
                sr.next = concat(ra>=0x80, ra==0, sr[6:0])
            elif rw == 1:
                sr.next = concat(rx>=0x80, rx==0, sr[6:0])
            if ir == 0x17:
                pc.next = concat(data_out[3:], pc[8:])
                adr.next = concat(data_out, pc[8:])
            else:
                adr.next = pc
            we.next = 0
            rw.next = 0
            cyc.next = s.F1
        else:
            cyc.next = s.F1 # Should be an exception

    return instances()


BoolOp(op=And(), values=[Compare(left=Name(id='ir', ctx=Load()), ops=[Eq()], comparators=[Attribute(value=Name(id='opc', ctx=Load()), attr='LDA', ctx=Load())]), Compare(left=Name(id='am', ctx=Load()), ops=[Eq()], comparators=[Attribute(value=Name(id='adm', ctx=Load()), attr='ABS', ctx=Load())])])


In [17]:
from myirl.targets import VHDL
from myirl.test import common_test

def test():
    clk = ClkSignal()
    rst = ResetSignal(ResetSignal.POS_ASYNC)
    data_in, data_out = [ Signal(intbv()[8:]) for _ in range(2) ]
    adr = Signal(intbv()[16:])
    we = Signal(bool())
    
    inst = processor(clk, rst, data_in, data_out, adr, we)
    files = inst.elab(targets.VHDL)
    common_test.run_ghdl(files, inst, debug = True)
test()

Function call: 'processor(clk=clk,rst=rst,data_in=data_in,data_out=data_out,adr=adr,we=we)'
 Module top_processor: Existing implementation processor, rename to processor_2 
Creating process 'processor/assign' with sensitivity ()
Creating process 'processor/logic' with sensitivity (<clk>,)
 Insert unit processor/_s1_s1_s8_s8_s16_s1 
 DEBUG Inline blackbox [Component 'bshift/bshift'] 
 use default parameter SHIFT_RIGHT : False 
 DEBUG Inline blackbox [Component 'bshift/bshift'] 
 Set parameter SHIFT_RIGHT := True 
 DEBUG: Writing 'processor_2' to file /tmp/processor_2.vhdl 
Finished _elab in 0.0043 secs
==== COSIM stdout ====

==== COSIM stderr ====

==== COSIM stdout ====

==== COSIM stderr ====
/tmp/processor_2.vhdl:246:5: identifier "inst_bshift_0" already used for a declaration
/tmp/processor_2.vhdl:229:5: previous declaration: component instance "inst_bshift_0"
/tmp/processor_2.vhdl:253:5: identifier "inst_bshift_1" already used for a declaration
/tmp/processor_2.vhdl:236:5: previou

../../../myirl/kernel/components.py:1135: UserWarning: Implicit sign conversion <jumppc> <= ADD_builtin(pc, SGN(<im>))
  (v, type(v), sz, target_size))


AnalysisError: Failed

In [15]:
!cat /tmp/processor.vhdl

-- File generated from /usr/local/lib/python3.8/runpy.py
-- (c) 2016-2021 section5.ch
-- Modifications may be lost

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.txt_util.all;
use work.myirl_conversion.all;

entity processor is
    port (
        clk : in std_ulogic;
        rst : in std_ulogic;
        data_in : out unsigned(7 downto 0);
        data_out : in unsigned(7 downto 0);
        adr : out unsigned(15 downto 0);
        we : out std_ulogic
    );
end entity processor;

architecture MyIRL of processor is
    -- Local type declarations
    type e_870c is (
        e_870c_F1,
        e_870c_F2,
        e_870c_D,
        e_870c_E,
        e_870c_M1,
        e_870c_M2
    );
    -- Signal declarations
    signal incim : unsigned(7 downto 0);
    signal incpc : unsigned(10 downto 0);
    signal incpc2 : unsigned(10 downto 0);
    signal pushsp : unsigned(7 downto 0);
    signal popsp : unsigned(7 downto 0);
    signal jumppc : unsigne